# Process True False Dataset

Dataset from the paper [The Internal State of an LLM Knows When It's Lying](https://aclanthology.org/2023.findings-emnlp.68.pdf)

You can download the dataset [here](http://azariaa.com/Content/Datasets/true-false-dataset.zip).

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import minimize_scalar
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

device = "cuda" if torch.cuda.is_available() else "CPU"
print(f'Device: {device}')

In [ ]:
dataset_name = 'true-false-dataset'
categories = ['animals', 'cities', 'companies', 'elements', 
              'facts', 'generated', 'inventions']

data_path = f'../Data/{dataset_name}'
embeddings_path = f'Embeddings/{dataset_name}'
concepts_path = f'Concepts/{dataset_name}'
cos_sims_path = f'Cosine_Similarities/{dataset_name}'
images_path = f'Images/{dataset_name}'

In [ ]:
embeddings_file = os.path.join(embeddings_path, 'embeddings.pt')
    
if os.path.exists(embeddings_file):
    print('Embeddings file found. No need to load model and tokenizer.')
    model = None
    tokenizer = None
else:
    print('Embeddings file NOT found. Loading model and tokenizer...')
    
    from huggingface_hub import notebook_login
    notebook_login()
    
    model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
    model.to(device)
    model.eval()
    print('Done.')

## Processing statements

In [ ]:
def get_category(category, data_path):
    cat_path = os.path.join(data_path, f'{category}_true_false.csv')
    df_cat = pd.read_csv(cat_path)
    df_cat[category] = 1
    return df_cat

In [ ]:
def get_categories(categories, data_path):
    df_list = []
    for category in categories:
        df_cat = get_category(category, data_path=data_path)
        df_list.append(df_cat)
    df = pd.concat(df_list, ignore_index=True)
    df = df.fillna(0)
    for category in categories:
        df[category] = df[category].astype(int)
    df = df.rename(columns={'label': 'true'})
    return df

In [ ]:
def get_hidden_states(statements, model, tokenizer, embeddings_path, 
                      device=device, save=True):
    embeddings_file = os.path.join(embeddings_path, 'embeddings.pt')
    
    if os.path.exists(embeddings_file):
        print('   Hidden states file found.')
        hidden_states = torch.load(embeddings_file)
    else:
        print('   Hidden states file NOT found.')
        hidden_states = []
        for statement in tqdm(statements, desc='Getting hidden states'):
            tokenized_prompt = tokenizer(statement, return_tensors="pt").to(device)
            output = model(**tokenized_prompt, output_hidden_states=True)
            
            features = output.hidden_states[-1][0][-1]
            hidden_state = features.cpu().detach()
            hidden_states.append(hidden_state)
            
            del output
            del features
            torch.cuda.empty_cache()

        hidden_states = torch.stack(hidden_states, dim=0)
        # Center embeddings
        average_embedding = torch.mean(hidden_states, 0) 
        centered_hidden_states = hidden_states - average_embedding
        if save:
            os.makedirs(embeddings_path, exist_ok=True)
            torch.save(hidden_states, embeddings_file)
            
    return hidden_states

In [ ]:
def process_data(categories, model, tokenizer, data_path, embeddings_path):
    print('Getting statements...', end=' ')
    df = get_categories(categories, data_path=data_path)
    df.to_csv(os.path.join(data_path, 'metadata.csv'), index=False)
    statements = df['statement'].to_list()
    print('Done.')

    print('Getting hidden states...')
    hidden_states = get_hidden_states(statements, model, tokenizer, embeddings_path)
    print('Done.')

    return df, hidden_states

In [ ]:
df, hidden_states = process_data(categories, model, tokenizer, data_path, embeddings_path)

In [ ]:
print(f'Number of statements: {df.shape[0]}')
print(f'Number of true statements: {df[df["true"] == 1].shape[0]}')
print(f'Number of false statements: {df[df["true"] == 0].shape[0]}')

## Computing concepts

In [ ]:
def compute_single_concept(concept, df, hidden_states):
    # Get embeddings from the concept
    hidden_states_cat = hidden_states[df[concept]==1] 
    # Get average embedding
    concept_vect = torch.mean(hidden_states_cat, 0) 
    # Normalize vector
    concept_vect = concept_vect / concept_vect.norm() 
    return concept_vect

def compute_concepts(df, hidden_states, concepts_path, save=True):
    concept_names = list(df.columns[1:])
    concepts = {}

    print('Computing concept vectors...', end=' ')
    for concept in concept_names:
        concept_vect = compute_single_concept(concept, df, hidden_states)
        concepts[concept] = concept_vect
    print('Done.')
        
    if save:
        print('Saving concepts...', end=' ')
        os.makedirs(concepts_path, exist_ok=True)
        concepts_path = os.path.join(concepts_path, 'concepts.pt')
        torch.save(concepts, concepts_path)
        print('Done.')

    print(f'Concepts computed: {list(concepts.keys())}')
    return concepts

In [ ]:
concepts = compute_concepts(df, hidden_states, concepts_path)

## Compute cosine similarities

In [ ]:
def compute_cosine_sims(hidden_states, concept_embs, concept_names, cos_sims_path,
                        save=True):
    print('Computing cosine similarities...', end=' ')
    similarities = cosine_similarity(hidden_states, concept_embs)
    sim_df = pd.DataFrame(similarities, columns=concept_names)
    print('Done.')
    if save:
        print('Saving cosine similarities...', end=' ')
        os.makedirs(cos_sims_path, exist_ok=True)
        cos_sims_df_path = os.path.join(cos_sims_path, 'cosine_similarities.csv')
        sim_df.to_csv(cos_sims_df_path, index=False)
        print('Done.')
        
    return sim_df

In [ ]:
concept_embs = torch.stack(list(concepts.values()), dim=0)
concept_names = list(concepts.keys())
sim_df = compute_cosine_sims(hidden_states, concept_embs, concept_names, cos_sims_path)

In [ ]:
def plot_cos_sim_histograms(df, sim_df, images_path):
    concept_names = list(df.columns[1:])
    fig, axs = plt.subplots(2, 4, figsize=(8,4), sharex=True)
    bins = np.linspace(0, 1, 20)
    for i, concept in enumerate(concept_names):
        ix, iy = int(i/4), i%4
        sns.kdeplot(sim_df[df[concept]==1][concept], 
                    label='In concept', ax=axs[ix,iy])
        sns.kdeplot(sim_df[df[concept]==0][concept], 
                    label='Out of concept', ax=axs[ix,iy])
        axs[ix,iy].set_xlabel('')
        axs[ix,iy].set_ylabel('')
        axs[ix,iy].set_title(concept)
    
    handles, labels = axs[0,0].get_legend_handles_labels()
    lgd = fig.legend(handles, labels, bbox_to_anchor=(1.2, 0.9))
    xl = fig.supxlabel('Cosine Similarity')
    yl = fig.supylabel('Proportion of samples')
    title = fig.suptitle('Cosine similarity in/out of concept')
    fig.tight_layout()

    print('Saving image...', end=' ')
    os.makedirs(images_path, exist_ok=True)
    cos_sims_image_path = os.path.join(images_path, 'cosine_similarities.png')
    fig.savefig(cos_sims_image_path, bbox_extra_artists=(lgd,xl,yl,title), 
                bbox_inches='tight')
    print('Done.')
    
    plt.show()

In [ ]:
plot_cos_sim_histograms(df, sim_df, images_path)

In [ ]:
def show_similarity_concepts(concepts, images_path):
    concept_embs = torch.stack(list(concepts.values()), dim=0)
    concept_names = list(concepts.keys())
    cos_sim_concepts_df = compute_cosine_sims(concept_embs, 
                                              concept_embs, 
                                              concept_names, 
                                              '', save=False)
    cos_sim_concepts_df['Concept'] = list(concepts.keys())
    cos_sim_concepts_df = cos_sim_concepts_df.set_index('Concept')
    ax = sns.heatmap(cos_sim_concepts_df, annot=True, fmt=".2f", cmap="Blues",
               cbar_kws={'label': 'Cosine similarity'})
    ax.set_title('Cosine similarity between concepts')

    print('Saving image...', end=' ')
    os.makedirs(images_path, exist_ok=True)
    cos_sims_image_path = os.path.join(images_path, 'cosine_similarities_concepts.png')
    plt.savefig(cos_sims_image_path)
    print('Done.')
    
    plt.show()

In [ ]:
show_similarity_concepts(concepts, images_path)

## Threshold-based models

In [ ]:
class ThresholdModel:
    def __init__(self):
        pass

    def predict(self, X_eval, thresh=None):
        if thresh is None:
            thresh = self.thresh
        return (X_eval[:,0] > thresh).astype(int)

    def loss(self, thresh):
        y_pred = self.predict(self.X, thresh)
        error = 1 - accuracy_score(self.y, y_pred)
        return error
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        result = minimize_scalar(self.loss, bounds=(-1, 1), method='bounded')
        self.thresh, self.loss = result.x, result.fun
        return self.thresh, self.loss

def train_test_threshold(X, y, stratify_list=None):
    model = ThresholdModel()
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.33, 
                                                        random_state=42,
                                                        stratify=stratify_list)
    thresh, train_error = model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test, thresh)
    test_error = 1 - accuracy_score(y_test, y_test_pred)
    return thresh, train_error, test_error

def get_concept_X_y_list(df, sim_df, concept):
    X = sim_df[[concept]].to_numpy()
    y = (df[concept]==1).to_numpy().astype(int)
    concept_list = df[concept]
    return X, y, concept_list

In [ ]:
def get_global_threshold(df, sim_df, verbose=True):
    X_list = []
    y_list = []
    concept_lists = []
    concepts = list(sim_df.columns)
    for concept in concepts:
        X, y, concept_list = get_concept_X_y_list(df, sim_df, concept)
        X_list.append(X)
        y_list.append(y)
        concept_lists.append(concept_list)
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list)
    concept_list = np.concatenate(concept_lists)

    thresh, train_error, test_error = train_test_threshold(X, y, stratify_list=concept_list)
    if verbose:
        print(f'Global threshold: {thresh:.3f}')
        print(f'Train error: {train_error:.3f} | Test error: {test_error:.3f}')
    return thresh, train_error, test_error

In [ ]:
global_thresh, global_train_error, global_test_error = get_global_threshold(df, sim_df)

In [ ]:
def get_concept_threshold(df, sim_df, concept):
    X, y, concept_list = get_concept_X_y_list(df, sim_df, concept)
    thresh, train_error, test_error = train_test_threshold(X, y, stratify_list=concept_list)
    return thresh, train_error, test_error

def get_individual_thresholds(df, sim_df, verbose=True):
    concepts = list(sim_df.columns)
    thresholds = {}
    train_errors = {}
    test_errors = {}
    for concept in concepts:
        thresh, train_error, test_error = get_concept_threshold(df, sim_df, concept)
        thresholds[concept] = thresh
        train_errors[concept] = train_error
        test_errors[concept] = test_error
        if verbose:
            print(f'Concept: {concept.ljust(10)} | Threshold: {thresh:.3f} | Train error: {train_error:.3f} | Test error: {test_error:.3f}')
    return thresholds, train_errors, test_errors

In [ ]:
thresholds, train_errors, test_errors = get_individual_thresholds(df, sim_df)

In [ ]:
def plot_concept_thresh_errors(test_errors):
    fig, ax = plt.subplots(1,1)
    sns.barplot(test_errors, orient="y", ax=ax)
    ax.set_xlabel('Classification error')
    ax.set_ylabel('Concept')
    ax.set_title('Classification error of concept-specific threshold')
    plt.show()

In [ ]:
plot_concept_thresh_errors(test_errors)